# Machine Learning in Julia: Flux.jl

<img src="https://fluxml.ai/logo.png" width=800>

<img src="flux.png" width=900>

Web page: https://fluxml.ai/

Examples: [Model zoo](https://github.com/FluxML/model-zoo/)

# A single neuron

In [1]:
using Flux

┌ Info: Recompiling stale cache file C:\Users\carsten\.julia\compiled\v1.1\Flux\QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184


In [2]:
model(W,b,x) = σ.(W * x + b)

model (generic function with 1 method)

In [3]:
# single neuron 5 in 1 out
W = randn(1, 5) # weights
b = zeros(1)    # biases
x = rand(5)     # input

5-element Array{Float64,1}:
 0.5160287138043114 
 0.9829057492794211 
 0.15298584819665506
 0.775591839600053  
 0.13052244878138985

In [4]:
model(W, b, x)

1-element Array{Float64,1}:
 0.529861161413611

In [5]:
loss(W, b, x) = Flux.mse(model(W,b,x), 0.5)

loss (generic function with 1 method)

In [6]:
loss(W,b,x)

0.0008916889609697274

In [7]:
import Flux.Tracker: gradient # AD

gradient(loss, W, b, x)

([0.00767713 0.014623 … 0.0115387 0.00194183] (tracked), [0.0148773] (tracked), [0.00716614, -0.00376108, 0.0140566, -0.00306655, 0.0153683] (tracked))

Since there can be hundreds of parameters in a neural network, we use a slightly different approach.

In [9]:
using Flux.Tracker: param, back!, grad

W = param(randn(1, 5))
b = param(zeros(1))
x = rand(5)

y = loss(W, b, x)

back!(y) # Automatic differentiation (backpropagation)

grad(W), grad(b)

([0.0460407 0.0247984 … 0.0329554 0.0321991], [0.0473799])

We can now use these gradients to update our parameters.

In [10]:
using Flux.Tracker: update!

η = 0.1
for p in (W, b)
  update!(p, -η * grad(p)) # gradient descent
end

Of course, Flux offers more sophisticated optimizers, like [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) etc.

# A small Neural Network

In [14]:
m = Chain(
    Dense(10, 5),
    Dense(5, 2),
    softmax # normalize output neurons
)

opt = ADAM(0.01)

data, labels = rand(10, 100), fill(0.5, 2, 100)

loss(x, y) = sum(Flux.mse(m(x), y))

Flux.train!(loss, params(m), [(data,labels)], opt)

In [15]:
m(rand(10)) # trained model

Tracked 2-element Array{Float32,1}:
 0.58191997f0
 0.41808006f0

# Learning the Ising transition

Loading packages

In [26]:
using Flux
using Flux: crossentropy, onecold, onehotbatch, throttle, @epochs
using Printf, Statistics, Random
using Base.Iterators: repeated

Data preparation

In [27]:
# our (fake) Monte Carlo configurations
confs_left = rand(64,4000)
confs_right = rand(64,4000)

# set up as training data
neach = size(confs_left, 2)
X = hcat(confs_left, confs_right)
labels = vcat(fill(1, neach), fill(0, neach))
Y = onehotbatch(labels, 0:1)
dataset = repeated((X, Y), 10)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}(([0.317564 0.460574 … 0.986012 0.884535; 0.286774 0.631787 … 0.849444 0.637524; … ; 0.873912 0.302887 … 0.401733 0.693611; 0.050445 0.195238 … 0.64096 0.557009], Bool[false false … true true; true true … false false])), 10)

Neural network + Training

In [29]:
# create neural network with 10 hidden units and 2 output neurons
m = Chain(
  Dense(64, 10, relu),
  Dense(10, 2),
  softmax)

# define cost-function
loss(x, y) = crossentropy(m(x), y)
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

opt = ADAM()

println("-------- Training")
@epochs 10 Flux.train!(loss, params(m), dataset, opt)

-------- Training


┌ Info: Epoch 1
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 2
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 3
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 4
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 5
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 6
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 7
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 8
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 9
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
┌ Info: Epoch 10
└ @ Main C:\Users\carsten\.julia\packages\Flux\qXNjB\src\optimise\train.jl:105
